<a href="https://colab.research.google.com/github/Ertyuuu55/Spacely/blob/main/APP_PY_SPACELY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instal Streamlit dan Pandas (dan paket lain yang Anda perlukan)
!pip install streamlit pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 132.6 MB/s eta 0:00:00


In [2]:
import streamlit as st
import pandas as pd

# 1. Sample DataFrame (mimicking df_streamlit from the notebook)
# This DataFrame is a simplified version for demonstration in Streamlit.
# In a real application, this would be loaded from a file or database.
data = {
    'price': [52.08, 52.48, 53.13, 54.14, 55.67, 59.03, 62.10, 65.25, 45.00, 48.50, 49.00, 50.70, 51.20, 70.00, 75.00, 80.00, 85.00, 90.00, 95.00, 100.00, 105.00, 110.00, 115.00, 120.00, 125.00, 130.00, 135.00, 140.00, 145.00, 150.00],
    'category': ['Bed', 'Desk', 'Desk', 'Desk', 'Desk', 'Desk', 'Chair', 'Chair', 'Table', 'Table', 'Table', 'Sofa', 'Sofa', 'Bed', 'Chair', 'Desk', 'Table', 'Sofa', 'Bed', 'Chair', 'Desk', 'Table', 'Sofa', 'Bed', 'Chair', 'Desk', 'Table', 'Sofa', 'Bed', 'Chair'],
    'material': ['Wood', 'Metal', 'Glass', 'Plastic', 'Wood', 'Metal', 'Plastic', 'Wood', 'Glass', 'Metal', 'Plastic', 'Wood', 'Metal', 'Glass', 'Plastic', 'Wood', 'Metal', 'Plastic', 'Wood', 'Metal', 'Glass', 'Plastic', 'Wood', 'Metal', 'Glass', 'Plastic', 'Wood', 'Metal', 'Glass', 'Plastic'],
    'color': ['Brown', 'Black', 'White', 'Grey', 'Brown', 'Black', 'Red', 'Blue', 'Green', 'Yellow', 'Black', 'White', 'Grey', 'Brown', 'Black', 'White', 'Grey', 'Red', 'Blue', 'Green', 'Yellow', 'Black', 'White', 'Grey', 'Brown', 'Black', 'White', 'Grey', 'Red', 'Blue']
}
df = pd.DataFrame(data)

# 2. Consolidated Prioritized Greedy Selection Function (copied from notebook)
def consolidated_prioritized_greedy_selection_with_quantities(df, budget, desired_categories_with_quantities):
    selected_items = []
    total_cost = 0.0
    selection_messages = []

    # Create a copy of the DataFrame to manage available items
    remaining_df = df.copy()

    # 1. Prioritized selection based on desired categories and quantities
    for item_request in desired_categories_with_quantities:
        category = item_request['category']
        quantity_desired = item_request['quantity']
        selected_count_for_category = 0

        if category not in df['category'].unique(): # Check if category exists in original df
            selection_messages.append(f"Warning: Desired category '{category}' is not available in our inventory.")
            continue

        category_items_initial = remaining_df[remaining_df['category'] == category].sort_values(by='price', ascending=True)

        if category_items_initial.empty:
            selection_messages.append(f"Warning: No items found for desired category '{category}' currently in stock.")
            continue

        for _ in range(quantity_desired):
            # Re-filter category items from the current remaining_df to get the cheapest available
            current_category_items = remaining_df[remaining_df['category'] == category].sort_values(by='price', ascending=True)

            if not current_category_items.empty:
                cheapest_item = current_category_items.iloc[0]
                if total_cost + cheapest_item['price'] <= budget:
                    selected_items.append(cheapest_item.to_dict())
                    total_cost += cheapest_item['price']
                    # Remove the selected item from remaining_df by its index
                    remaining_df = remaining_df.drop(cheapest_item.name)
                    selected_count_for_category += 1
                else:
                    selection_messages.append(f"Could not afford remaining {quantity_desired - selected_count_for_category} item(s) from category '{category}' (cheapest available: ${cheapest_item['price']:.2f}) within budget. Budget remaining: ${budget - total_cost:.2f}")
                    break # Break from the inner loop for this category
            else:
                selection_messages.append(f"Ran out of available items for category '{category}' before fulfilling {quantity_desired} items. Selected {selected_count_for_category}.")
                break

        if selected_count_for_category == quantity_desired:
            selection_messages.append(f"Successfully selected {quantity_desired} items for category '{category}'.")
        elif selected_count_for_category > 0:
            selection_messages.append(f"Selected {selected_count_for_category} out of {quantity_desired} desired items for category '{category}'.")

    # 2. Greedily select other cheapest items from the remaining budget and DataFrame
    selection_messages.append("Attempting to fill remaining budget with other cheapest items.")
    sorted_remaining_df = remaining_df.sort_values(by='price', ascending=True)

    for index, row in sorted_remaining_df.iterrows():
        item_price = row['price']
        if total_cost + item_price <= budget:
            selected_items.append(row.to_dict())
            total_cost += item_price
        else:
            # If adding the current item exceeds the budget, stop.
            selection_messages.append(f"Stopped greedy selection as next item (price: ${item_price:.2f}) exceeds remaining budget (${budget - total_cost:.2f}).")
            break

    if not selected_items:
        selection_messages.append("No items were selected within the budget criteria.")

    return selected_items, total_cost, selection_messages

# 3. Streamlit Application UI
st.set_page_config(layout="centered", page_title="Furniture Recommender")
st.title("Furniture Recommendation System")
st.write("Enter your budget and desired furniture categories with quantities to get recommendations.")

st.header("User Inputs")

# Input for budget
user_budget = st.number_input("Enter your total budget ($):", min_value=0.0, value=300.0, step=10.0)

# Get unique categories from the DataFrame for display
available_categories_str = ', '.join(df['category'].unique())

# Input for categories and quantities
user_categories_quantities_input = st.text_input(
    f"Enter desired furniture categories with quantities (e.g., Bed:1, Chair:2). Options: {available_categories_str}",
    value="Bed:1, Desk:5"
)

# Process user categories and quantities input
user_desired_categories_with_quantities = []
if user_categories_quantities_input:
    for item_str in user_categories_quantities_input.split(','):
        item_str = item_str.strip()
        if not item_str:
            continue

        parts = item_str.split(':')
        if len(parts) == 2:
            category = parts[0].strip()
            try:
                quantity = int(parts[1].strip())
                if category in df['category'].unique():
                    user_desired_categories_with_quantities.append({"category": category, "quantity": quantity})
                else:
                    st.warning(f"Warning: Category '{category}' is not a valid option and will be ignored.")
            except ValueError:
                st.warning(f"Warning: Quantity for '{category}' is not a valid number and will be ignored. Please enter an integer.")
        else:
            st.warning(f"Warning: Invalid input format for '{item_str}'. Please use 'Category:Quantity'.")

# Button to trigger recommendations
if st.button('Generate Recommendations'):
    if not user_desired_categories_with_quantities and user_budget == 0:
        st.error("Please enter a budget or desired categories.")
    else:
        st.subheader("Recommendation Results")

        recommended_furniture, final_cost, final_messages = consolidated_prioritized_greedy_selection_with_quantities(
            df, user_budget, user_desired_categories_with_quantities
        )

        for msg in final_messages:
            st.info(msg)

        if recommended_furniture:
            st.write(f"\nRecommended Furniture (total items: {len(recommended_furniture)}):")
            # Convert list of dicts to DataFrame for better display in Streamlit
            recommended_df = pd.DataFrame(recommended_furniture)
            st.dataframe(recommended_df[['category', 'price', 'material', 'color']].style.format({
                'price': "${:.2f}"
            }))
            st.markdown(f"**Total cost of selected furniture: ${final_cost:.2f}**")
        else:
            st.write("No furniture could be recommended based on your criteria.")


2025-12-02 07:29:59.532 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-02 07:29:59.533 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-02 07:29:59.645 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-12-02 07:29:59.646 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-02 07:29:59.646 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-02 07:29:59.648 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-02 07:29:59.649 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

In [3]:
# 1. Instal localtunnel untuk membuat terowongan publik
!pip install localtunnel

# 2. Jalankan aplikasi Streamlit di latar belakang (&) pada port 8501
!streamlit run app.py &

# 3. Buat terowongan ke port 8501 (port default Streamlit)
# Ini akan menampilkan link URL yang bisa Anda klik.
!lt --port 8501

ERROR: Could not find a version that satisfies the requirement localtunnel (from versions: none)
ERROR: No matching distribution found for localtunnel
Usage: streamlit run [OPTIONS] [TARGET] [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py
/bin/bash: line 1: lt: command not found
